In [ ]:
#|default_exp embeddings

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#|hide
from nbdev.showdoc import *

In [ ]:
#| export

from clip_plot.utils import timestamp, clean_filename
from clip_plot.images import image_to_array, Image

import os
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import preprocess_input

from tqdm.autonotebook import tqdm
import numpy as np


# Create and/or load embeddings

In [ ]:
#| export

def get_inception_vectors(**kwargs):
    """Create and return Inception vector representation of Image() instances"""
    print(
        timestamp(),
        "Creating Inception vectors for {} images".format(len(kwargs["image_paths"])),
    )
    vector_dir = os.path.join(kwargs["out_dir"], "image-vectors", "inception")
    if not os.path.exists(vector_dir):
        os.makedirs(vector_dir)
    base = InceptionV3(
        include_top=True,
        weights="imagenet",
    )
    model = Model(inputs=base.input, outputs=base.get_layer("avg_pool").output)
    print(timestamp(), "Creating image array")
    vecs = []
    with tqdm(total=len(kwargs["image_paths"])) as progress_bar:
        for idx, i in enumerate(Image.stream_images(image_paths=kwargs["image_paths"], metadata=kwargs["metadata"])):
            vector_path = os.path.join(vector_dir, clean_filename(i.path) + ".npy")
            if os.path.exists(vector_path) and kwargs["use_cache"]:
                vec = np.load(vector_path)
            else:
                im = preprocess_input(image_to_array(i.original.resize((299, 299))))
                vec = model.predict(np.expand_dims(im, 0)).squeeze()
                np.save(vector_path, vec)
            vecs.append(vec)
            progress_bar.update(1)
    return np.array(vecs)

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()